# [cknowledge.org/ai](http://cknowledge.org/ai): Crowdsourcing benchmarking and optimization of AI

* [Reproducible Quality-Efficient Systems Tournaments](http://cknowledge.org/request) ([ReQuEST initiative](http://cknowledge.org/request.html#organizers))
* [AI artifacts](http://cknowledge.org/ai-artifacts) (cTuning foundation)
* [Android app](https://play.google.com/store/apps/details?id=openscience.crowdsource.video.experiments) (dividiti)
* [Desktop app](https://github.com/dividiti/ck-crowdsource-dnn-optimization) (dividiti)
* [CK-Caffe](https://github.com/dividiti/ck-caffe) (Berkeley)
* [CK-Caffe2](https://github.com/ctuning/ck-caffe2) (Facebook)
* [CK-CNTK](https://github.com/ctuning/ck-cntk) (Microsoft)
* [CK-KaNN](https://github.com/dividiti/ck-kann) (Kalray)
* [CK-MVNC](https://github.com/ctuning/ck-mvnc) (Movidius / Intel)
* [CK-MXNet](https://github.com/ctuning/ck-mxnet) (Apache)
* [CK-NNTest](https://github.com/ctuning/ck-nntest) (cTuning foundation)
* [CK-TensorFlow](https://github.com/ctuning/ck-tensorflow) (Google)
* [CK-TensorRT](https://github.com/dividiti/ck-tensorrt) (NVIDIA)
* etc.

# [dividiti](http://dividiti.com)'s submission to [ReQuEST @ ASPLOS'18](http://cknowledge.org/request-cfp-asplos2018.html)

## Table of Contents

1. [Overview](#overview)
1. [Platforms](#platforms)
  1. [Linaro HiKey960](#platforms_hikey) (**"HiKey"**)
  1. [Firefly RK3399](#platforms_firefly) (**"Firefly"**)
1. [Experimental data](#data) [for developers]
1. [Data wrangling code](#code) [for developers]
1. [Experiments on Hikey](#experiments_hikey)
   1. [TensorFlow](#experiments_tensorflow_hikey)
   1. [ArmCL](#experiments_armcl_hikey)
   1. [ArmCL vs. TensorFlow](#experiments_armcl_tensorflow_hikey)
1. [Experiments on Firefly](#experiments_firefly)
   1. [TensorFlow](#experiments_tensorflow_firefly)
   1. [ArmCL](#experiments_armcl_firefly)
   1. [ArmCL vs. TensorFlow](#experiments_armcl_tensorflow_firefly)

<a id="overview"></a>
## Overview

This Jupyter Notebook studies performance (execution time) vs accuracy (top1 / top5) using the [Arm Compute Library](https://github.com/ARM-software/ComputeLibrary) on two development platforms:
- [Linaro HiKey960](https://www.96boards.org/product/hikey960/);
- [Firefly RK3399](http://en.t-firefly.com/index.php/product/rk3399.html).

<a id="platforms"></a>
## Platforms

<a id="platforms_hikey"></a>
### Linaro HiKey960

  - Chip:
     - [HiSilicon Kirin 960](http://www.hisilicon.com/en/Solutions/Kirin)
  - CPU ("performance" / "big"):
    - ARM&reg; Cortex&reg;-A73;
    - Max clock 2362 MHz;
    - 4 cores;
  - CPU ("efficiency" / "LITTLE"):
    - ARM&reg; Cortex&reg;-A53;
    - Max clock 1844 MHz;
    - 4 cores;
  - GPU:
    - ARM&reg; Mali&trade; G71 architecture;
    - Max clock 1037 MHz;
    - 8 cores;
    - OpenCL driver (`hikey962`: `instr=1,clexperimental=1,softjobpatch`):
```
$ ck run program:tool-print-opencl-devices | grep "version:"
OpenCL 2.0 v1.r6p0-01rel0.24c5f5e966f2b7f1f19b91d6f32ff53e
```

  - RAM:
    - LPDDR4 SDRAM;
    - 3 GB;

  - BSP:
    - Debian Stretch (9) Linux
```
$ uname -a
Linux hikey962 4.4.74-00216-g10816f6 #3 SMP PREEMPT Thu Jul 6 14:38:42 BST 2017 aarch64 GNU/Linux
```

In [ ]:
hikey_model = 'HiKey960\x00'
hikey_name  = 'Linaro HiKey960'
hikey_id    = 'hikey-960'
hikey_gpu   = 'Mali-G71 MP8'
hikey_gpu_mhz = '807 MHz'

<a id="platforms_firefly"></a>
### Firefly RK3399

  - Chip:
    - [Rockchip RK3399](http://rockchip.wikidot.com/rk3399)
  - CPU ("big"):
    - ARM&reg; Cortex&reg;-A72 architecture
    - Max clock 1800 MHz;
    - 2 cores;
  - CPU ("LITTLE"):
    - ARM&reg; Cortex&reg;-A53 architecture;
    - Max clock 1416 MHz;
    - 4 cores;
  - GPU:
    - ARM&reg; Mali&trade;-T860 architecture;
    - Max clock 800 MHz;
    - 4 cores;
    - OpenCL driver:
```
$ ck run program:tool-print-opencl-devices | grep "version:"
v1.r13p0-00rel0-git(a4271c9).31ba04af2d3c01618138bef3aed66c2c
```

  - RAM:
    - Samsung dual-channel DDR3;
    - 4 GB (8 GB swap);
  - BSP:
    - [Firefly-rk3399_xubuntu1604_201711301130.7z](https://drive.google.com/drive/u/0/folders/1lbaR7XVyHT4SnXkJ2ybj5YXAzAjDBWfT)
```
$ cat /etc/lsb-release
DISTRIB_ID=Ubuntu
DISTRIB_RELEASE=16.04
DISTRIB_CODENAME=xenial
DISTRIB_DESCRIPTION="Ubuntu 16.04.4 LTS"
$ uname -a
Linux firefly 4.4.77 #554 SMP Thu Nov 30 11:30:11 HKT 2017 aarch64 aarch64 aarch64 GNU/Linux
```

In [ ]:
firefly_model = 'Rockchip RK3399 Firefly Board (Linux Opensource)\x00'
firefly_name  = 'Firefly RK3399'
firefly_id    = 'firefly'
firefly_gpu   = 'Mali-T860 MP4'
firefly_gpu_mhz = '800 MHz'

### Platform mappings

In [ ]:
model_to_id = {
    firefly_model : firefly_id,
    hikey_model   : hikey_id
}
id_to_name = {
    firefly_id : firefly_name,
    hikey_id   : hikey_name
}
id_to_gpu = {
    firefly_id : firefly_gpu,
    hikey_id   : hikey_gpu
}
id_to_gpu_mhz = {
    firefly_id : firefly_gpu_mhz,
    hikey_id   : hikey_gpu_mhz
}

<a id="data"></a>
## Get the experimental data

The experimental data can be downloaded and registered with CK as described below.

### ArmCL experiments on HiKey

#### ArmCL accuracy experiments on 50,000 images

```
$ wget https://www.dropbox.com/s/tm1qlom7ehfbe0w/ck-request-asplos18-mobilenets-armcl-opencl-accuracy-50000.zip
$ ck add repo --zip=ck-request-asplos18-mobilenets-armcl-opencl-accuracy-50000.zip
```

In [ ]:
armcl_accuracy_50000_repo_uoa = 'ck-request-asplos18-mobilenets-armcl-opencl-accuracy-50000'
!ck list $armcl_accuracy_50000_repo_uoa:experiment:* | sort

#### ArmCL accuracy experiments on 500 images

```
$ wget https://www.dropbox.com/s/wqqchrhr36skm9y/ck-request-asplos18-mobilenets-armcl-opencl-accuracy-500.zip
$ ck add repo --zip=ck-request-asplos18-mobilenets-armcl-opencl-accuracy-500.zip
```

In [ ]:
armcl_accuracy_500_repo_uoa = 'ck-request-asplos18-mobilenets-armcl-opencl-accuracy-500'
!ck list $armcl_accuracy_500_repo_uoa:experiment:* | sort

#### ArmCL performance (latency) experiments

```
$ wget https://www.dropbox.com/s/wm3ahhm20y7g04k/ck-request-asplos18-mobilenets-armcl-opencl-performance.zip
$ ck add repo --zip=ck-request-asplos18-mobilenets-armcl-opencl-performance.zip
```

In [ ]:
armcl_performance_repo_uoa = 'ck-request-asplos18-mobilenets-armcl-opencl-performance'
!ck list $armcl_performance_repo_uoa:experiment:* | sort

### TensorFlow experiments on HiKey

#### TensorFlow accuracy experiments on 50000 images

```
$ wget https://www.dropbox.com/s/ro5txjz9n396s0t/ck-request-asplos18-mobilenets-tensorflow-accuracy-50000.zip
$ ck add repo --zip=ck-request-asplos18-mobilenets-tensorflow-accuracy-50000.zip
```

In [ ]:
tensorflow_accuracy_50000_repo_uoa = 'ck-request-asplos18-mobilenets-tensorflow-accuracy-50000'
!ck list $tensorflow_accuracy_50000_repo_uoa:experiment:* | sort

#### TensorFlow accuracy experiments on 500 images

```
$ wget https://www.dropbox.com/s/k0xhhb7owwvyfgu/ck-request-asplos18-mobilenets-tensorflow-accuracy-500.zip
$ ck add repo --zip=ck-request-asplos18-mobilenets-tensorflow-accuracy-500.zip
```

In [ ]:
tensorflow_accuracy_500_repo_uoa = 'ck-request-asplos18-mobilenets-tensorflow-accuracy-500'
!ck list $tensorflow_accuracy_500_repo_uoa:experiment:* | sort

#### TensorFlow performance (latency) experiments

```
$ wget https://www.dropbox.com/s/1fagdonfaqsdfou/ck-request-asplos18-mobilenets-tensorflow-performance.zip
$ ck add repo --zip=ck-request-asplos18-mobilenets-tensorflow-performance.zip
```

In [ ]:
tensorflow_performance_repo_uoa = 'ck-request-asplos18-mobilenets-tensorflow-performance'
!ck list $tensorflow_performance_repo_uoa:experiment:* | sort

### TensorFlow experiments on Firefly

#### TensorFlow accuracy experiments on 500 images

In [ ]:
firefly_tensorflow_accuracy_500_repo_uoa = 'ck-request-asplos18-mobilenets-tensorflow-accuracy-500-firefly'
!ck list $firefly_tensorflow_accuracy_500_repo_uoa:experiment:* | sort

#### TensorFlow performance (latency) experiments

In [ ]:
firefly_tensorflow_performance_repo_uoa = 'ck-request-asplos18-mobilenets-tensorflow-performance-firefly'
!ck list $firefly_tensorflow_performance_repo_uoa:experiment:* | sort

### ArmCL experiments on Firefly

#### ArmCL performance (latency) experiments

In [ ]:
firefly_armcl_performance_repo_uoa = 'ck-request-asplos18-mobilenets-armcl-opencl-performance-firefly'
!ck list $firefly_armcl_performance_repo_uoa:experiment:* | sort

#### ArmCL accuracy experiments on 500 images

In [ ]:
firefly_armcl_accuracy_500_repo_uoa = 'ck-request-asplos18-mobilenets-armcl-opencl-accuracy-500-firefly'
!ck list $armcl_accuracy_500_repo_uoa:experiment:* | sort

<a id="code"></a>
## Data wrangling code

**NB:** Please ignore this section if you are not interested in re-running or modifying this notebook.

### Includes

#### Standard

In [ ]:
import os
import sys
import json
import re

#### Scientific

If some of the scientific packages are missing, please install them using:
```
# pip install jupyter pandas numpy matplotlib
```

In [ ]:
import IPython as ip
import pandas as pd
import numpy as np
import matplotlib as mp
import seaborn as sb

In [ ]:
print ('IPython version: %s' % ip.__version__)
print ('Pandas version: %s' % pd.__version__)
print ('NumPy version: %s' % np.__version__)
print ('Matplotlib version: %s' % mp.__version__)
print ('Seaborn version: %s' % sb.__version__)

In [ ]:
from IPython.display import Image, display
def display_in_full(df):
    pd.options.display.max_columns = len(df.columns)
    pd.options.display.max_rows = len(df.index)
    display(df)

In [ ]:
import matplotlib.pyplot as plt
from matplotlib import cm
%matplotlib inline

In [ ]:
default_colormap = cm.autumn
default_fontsize = 16
default_barwidth = 0.8
default_figwidth = 24
default_figheight = 3
default_figdpi = 200
default_figsize = [default_figwidth, default_figheight]

In [ ]:
if mp.__version__[0]=='2': mp.style.use('classic')
mp.rcParams['figure.max_open_warning'] = 200
mp.rcParams['figure.dpi'] = default_figdpi
mp.rcParams['font.size'] = default_fontsize
mp.rcParams['legend.fontsize'] = 'medium'

#### Collective Knowledge

If CK is not installed, please install it using:
```
# pip install ck
```

In [ ]:
import ck.kernel as ck
print ('CK version: %s' % ck.__version__)

### Access experimental data

In [ ]:
def get_experimental_results(repo_uoa, tags='explore-mobilenets-performance', accuracy=False,
                             module_uoa='experiment', _library=None, _platform=None):
    r = ck.access({'action':'search', 'repo_uoa':repo_uoa, 'module_uoa':module_uoa, 'tags':tags})
    if r['return']>0:
        print('Error: %s' % r['error'])
        exit(1)
    experiments = r['lst']

    dfs = []
    for experiment in experiments:
        data_uoa = experiment['data_uoa']
        r = ck.access({'action':'list_points', 'repo_uoa':repo_uoa, 'module_uoa':module_uoa, 'data_uoa':data_uoa})
        if r['return']>0:
            print('Error: %s' % r['error'])
            exit(1)
        # Mapping of expected library tags to reader-friendly names.
        tag_to_name = {
            # ArmCL tags on HiKey.
            '17.12-48bc34ea'    : 'armcl-17.12',
            '18.01-f45d5a9b'    : 'armcl-18.01',
            '18.03-e40997bb'    : 'armcl-18.03',
            'request-d8f69c13'  : 'armcl-dv/dt', # armcl-18.03+
            '18.05-b3a371bc'    : 'armcl-18.05',
            # ArmCL tags on Firefly.
            '17.12-48bc34e'     : 'armcl-17.12',
            '18.01-f45d5a9'     : 'armcl-18.01',
            '18.03-e40997b'     : 'armcl-18.03',
            '18.05-b3a371b'     : 'armcl-18.05',
            # TensorFlow tags.
            'tensorflow-1.7'    : 'tensorflow-1.7',
            'tensorflow-1.8'    : 'tensorflow-1.8',
        }
            
        # Library.
        library_tags = [ tag for tag in r['dict']['tags'] if tag in tag_to_name.keys() ]
        if len(library_tags)==1:
            library = tag_to_name[library_tags[0]]
        else:
            print('[Warning] Bad library tags. Skipping experiment with tags:')
            print(r['dict']['tags'])
            continue
        if _library and _library!=library: continue
        # For each point.    
        for point in r['points']:
            point_file_path = os.path.join(r['path'], 'ckp-%s.0001.json' % point)
            with open(point_file_path) as point_file:
                point_data_raw = json.load(point_file)
            characteristics_list = point_data_raw['characteristics_list']
            num_repetitions = len(characteristics_list)
            platform = model_to_id[point_data_raw['features']['platform']['platform']['model']]
            if _platform and _platform!=platform: continue
            batch_size = np.int64(point_data_raw['choices']['env'].get('CK_BATCH_SIZE',-1))
            batch_count = np.int64(point_data_raw['choices']['env'].get('CK_BATCH_COUNT',-1))
            convolution_method = np.int64(point_data_raw['choices']['env'].get('CK_CONVOLUTION_METHOD_HINT',1))
            if library.startswith('tensorflow-'):
                multiplier = np.float64(point_data_raw['choices']['env'].get('CK_ENV_TENSORFLOW_MODEL_MOBILENET_MULTIPLIER',-1))
                resolution = np.int64(point_data_raw['choices']['env'].get('CK_ENV_TENSORFLOW_MODEL_MOBILENET_RESOLUTION',-1))
            else:
                multiplier = np.float64(point_data_raw['choices']['env'].get('CK_ENV_MOBILENET_WIDTH_MULTIPLIER',-1))
                resolution = np.int64(point_data_raw['choices']['env'].get('CK_ENV_MOBILENET_RESOLUTION',-1))
            model = 'v1-%.2f-%d' % (multiplier, resolution)
            if accuracy:
                data = [
                    {
                        # features
                        'platform': platform,
                        'library': library,
                        # choices
                        'model': model,
                        'batch_size': batch_size,
                        'batch_count': batch_count,
                        'convolution_method': convolution_method,
                        'resolution': resolution,
                        'multiplier': multiplier,
                        # statistical repetition
                        'repetition_id': repetition_id,
                        # runtime characteristics
                        'success?': characteristics['run'].get('run_success', 'n/a'),
                        'accuracy_top1': characteristics['run'].get('accuracy_top1', 0),
                        'accuracy_top5': characteristics['run'].get('accuracy_top5', 0),
                        'frame_predictions': characteristics['run'].get('frame_predictions', []),
#                         # recompute accuracy from frame_predictions (was incorrectly recorded in early experiments)
#                         'accuracy_top1_': len([
#                             prediction for prediction in characteristics['run'].get('frame_predictions', [])
#                             if prediction['accuracy_top1']=='yes'
#                         ]) / np.float64(batch_count),
#                         'accuracy_top5_': len([
#                             prediction for prediction in characteristics['run'].get('frame_predictions', [])
#                             if prediction['accuracy_top5']=='yes'
#                         ]) / np.float64(batch_count)
                    }
                    for (repetition_id, characteristics) in zip(range(num_repetitions), characteristics_list)
                ]
            else: # performance
                data = [
                    {
                        # features
                        'platform': platform,
                        'library': library,
                        # choices
                        'model': model,
                        'batch_size': batch_size,
                        'batch_count': batch_count,
                        'convolution_method': convolution_method,
                        'resolution': resolution,
                        'multiplier': multiplier,
                        # statistical repetition
                        'repetition_id': repetition_id,
                        # runtime characteristics
                        'success?': characteristics['run'].get('run_success', 'n/a'),
                        'time_avg_ms': characteristics['run']['prediction_time_avg_s']*1e+3,
                        'time_total_ms': characteristics['run']['prediction_time_total_s']*1e+3,
                    }
                    for (repetition_id, characteristics) in zip(range(num_repetitions), characteristics_list)
                ]
            index = [
                'platform', 'library', 'model', 'multiplier', 'resolution', 'batch_size', 'convolution_method', 'repetition_id'
            ]
            # Construct a DataFrame.
            df = pd.DataFrame(data)
            df = df.set_index(index)
            # Append to the list of similarly constructed DataFrames.
            dfs.append(df)
    if dfs:
        # Concatenate all thus constructed DataFrames (i.e. stack on top of each other).
        result = pd.concat(dfs)
        result.sort_index(ascending=True, inplace=True)
    else:
        # Construct a dummy DataFrame the success status of which can be safely checked.
        result = pd.DataFrame(columns=['success?'])
    return result

### Merge performance and accuracy data

In [ ]:
# Return a new DataFrame with only the performance and accuracy metrics.
def merge_performance_accuracy(df_performance, df_accuracy, 
                               reference_platform=None, reference_lib=None, reference_convolution_method=1,
                               performance_metric='time_avg_ms', accuracy_metric='accuracy_top1'):
    df = df_performance[[performance_metric]]
    accuracy_list = []
    for index, row in df.iterrows():
        (platform, lib, model, multiplier, resolution, batch_size, convolution_method) = index
        if reference_platform: platform = reference_platform
        try:
            accuracy = df_accuracy.loc[(platform, lib, model, multiplier, resolution, batch_size, convolution_method)][accuracy_metric]
        except:
            if reference_lib: lib = reference_lib
            convolution_method = reference_convolution_method
            accuracy = df_accuracy.loc[(platform, lib, model, multiplier, resolution, batch_size, convolution_method)][accuracy_metric]
        accuracy_list.append(accuracy)
    df = df.assign(accuracy_top1=accuracy_list) # FIXME: assign to the value of accuracy_metric
    return df

### Plot experimental data

In [ ]:
# 
def plot(df_performance_accuracy, libs=None, platform_id=hikey_id,
         performance_metric='time_avg_ms', accuracy_metric='accuracy_top1',
         xmin=0.0, xmax=75.1, xstep=5.0, ymin=0.4, ymax=0.751, ystep=0.05,
         title=None, save_fig=False, save_fig_name='mobilenets-default'):
    fig = plt.figure(figsize=(8,4), dpi=200)
    ax = fig.gca()
    
    lib_to_color = { 
        'armcl-17.12'    : 'red',
        'armcl-18.01'    : 'yellow',
        'armcl-18.03'    : 'orange',
        'armcl-dv/dt'    : 'green',
        'armcl-18.05'    : 'purple',
        'tensorflow-1.7' : 'cyan',
        'tensorflow-1.8' : 'blue',
    }
    multiplier_to_marker_0 = { 1.00 : '*', 0.75 : 'D', 0.50: 'v', 0.25 : '8' } # gemm
    multiplier_to_marker_1 = { 1.00 : 'p', 0.75 : 's', 0.50: '^', 0.25 : 'o' } # direct
    multiplier_to_marker_2 = { 1.00 : 'P', 0.75 : 'X', 0.50: '<', 0.25 : '.' } # winograd

    if libs==None: libs = df_performance_accuracy.index.levels[1].tolist()
    df = df_performance_accuracy.loc[platform_id].loc[libs]
    for index, row in df.iterrows():
        (lib, model, multiplier, resolution, batch_size, convolution_method) = index
        performance = row[performance_metric]
        accuracy = row[accuracy_metric]
        
        # Mark Pareto-optimal points.
        is_on_pareto = True
        for index1, row1 in df.iterrows():
            is_faster = row1[performance_metric] < row[performance_metric]
            is_no_less_accurate = row1[accuracy_metric] >= row[accuracy_metric]
            if is_faster and is_no_less_accurate:
                is_on_pareto = False
                break

        # GEMM-based convolution should be exactly the same in '18.03' and 'dv/dt', so plot
        # the minimum execution time of '18.03' and 'dv/dt' as '18.03'.
        if 'armcl-dv/dt' in libs and convolution_method==0 and (lib=='armcl-dv/dt' or lib=='armcl-18.03'):
            performance_dv_dt = df.loc[('armcl-dv/dt', model, multiplier, resolution, batch_size, convolution_method)][performance_metric]
            performance_18_03 = df.loc[('armcl-18.03', model, multiplier, resolution, batch_size, convolution_method)][performance_metric]
            if lib=='armcl-18.03':
                if (performance_dv_dt < performance_18_03):
                    continue
            if lib=='armcl-dv/dt':
                if (performance_dv_dt < performance_18_03):
                    lib = 'armcl-18.03' # change color
                else:
                    continue
        
        color = lib_to_color[lib]
        size = resolution / 16
        # Select marker for multiplier.
        if   convolution_method==0:
            marker = multiplier_to_marker_0[multiplier]
        elif convolution_method==1:
            marker = multiplier_to_marker_1[multiplier]
        elif convolution_method==2:
            marker = multiplier_to_marker_2[multiplier]
        else:
            marker = None

        # Plot.
        ax.plot(performance, accuracy, marker, markerfacecolor=color, markersize=size)

        # Mark Pareto-optimal points with scaled black pluses.
        if is_on_pareto:
            ax.plot(performance, accuracy, 'k+', markersize=size)

    # Title.
    if not title: title = '%s (GPU: %s @ %s)' % (id_to_name[platform_id], id_to_gpu[platform_id], id_to_gpu_mhz[platform_id])
    ax.set_title(title)
    # X axis.
    xlabel='Image recognition time (ms)' if performance_metric=='time_avg_ms' else ''
    ax.set_xlabel(xlabel)
    ax.set_xlim(xmin, xmax)
    ax.set_xticks(np.arange(xmin, xmax, xstep))
    for xtick in ax.xaxis.get_major_ticks(): xtick.label.set_fontsize(12)
    # Y axis.
    ylabel='Image recognition accuracy (top %s)' % accuracy_metric[-1]
    ax.set_ylabel(ylabel)
    ax.set_ylim(ymin, ymax)
    ax.set_yticks(np.arange(ymin, ymax, ystep))
    for ytick in ax.yaxis.get_major_ticks(): ytick.label.set_fontsize(12)
    # Legend.
    handles = [ 
        mp.patches.Patch(color=color, label=lib)
        for (lib, color) in sorted(lib_to_color.items())
        if lib in libs
    ]
    plt.legend(title='Library', handles=handles[::-1], loc='lower right')

    plt.grid()
    if save_fig:
        save_fig_path = os.path.join(save_fig_dir, '%s.%s' % (save_fig_name, save_fig_ext))
        plt.savefig(save_fig_path, dpi=default_figdpi, bbox_inches='tight')
    plt.show()

### Set options for saving figures/tables

In [ ]:
def get_paper_dir(module_uoa='dissemination.publication', data_uoa='08da9685582866a0'):
    r = ck.access({'action':'find','module_uoa':module_uoa,'data_uoa':data_uoa})
    if r['return']>0:
        print('Warning: %s' % r['error'])
        paper_dir = os.path.curdir
    else:
        paper_dir = r['path']
    return paper_dir

In [ ]:
save_fig_ext = 'pdf'
save_fig_dir = os.path.join(get_paper_dir(), 'figures')
if not os.path.exists(save_fig_dir):
    os.makedirs(save_fig_dir)

In [ ]:
save_tab = False
save_tab_ext = 'tex'
save_tab_dir = os.path.join(get_paper_dir(), 'tables')
if not os.path.exists(save_tab_dir):
    os.makedirs(save_tab_dir)

<a id="experiments_tensorflow_hikey"></a>
## TensorFlow experiments on HiKey

### TensorFlow performance (latency)

In [ ]:
df_tensorflow_performance_raw = get_experimental_results(repo_uoa=tensorflow_performance_repo_uoa,
                                                         tags='explore-mobilenets-performance', accuracy=False)
# Take the minimum execution time out of several repetitions.
df_tensorflow_performance = \
    df_tensorflow_performance_raw.groupby(level=df_tensorflow_performance_raw.index.names[:-1]).min()
# Display all rows and columns.
display_in_full(df_tensorflow_performance)

### TensorFlow accuracy on 500 images

In [ ]:
df_tensorflow_accuracy_500_raw = get_experimental_results(repo_uoa=tensorflow_accuracy_500_repo_uoa,
                                                          tags='explore-mobilenets-accuracy', accuracy=True)
# Extract frame predictions.
df_tensorflow_predictions_500 = df_tensorflow_accuracy_500_raw[['frame_predictions']]
# Reduce the repetition_id index dimension (only 1 repetition anyway).
df_tensorflow_accuracy_500 = \
    df_tensorflow_accuracy_500_raw[['accuracy_top1', 'accuracy_top5']] \
    .groupby(level=df_tensorflow_accuracy_500_raw.index.names[:-1]).min()
# Display all rows and columns.
display_in_full(df_tensorflow_accuracy_500)

### TensorFlow accuracy on 50,000 images (measured)

In [ ]:
df_tensorflow_accuracy_50000_raw = get_experimental_results(repo_uoa=tensorflow_accuracy_50000_repo_uoa,
                                                            tags='explore-mobilenets-accuracy', accuracy=True)
# Extract frame predictions.
df_tensorflow_predictions_50000 = df_tensorflow_accuracy_50000_raw[['frame_predictions']]
# Reduce the repetition_id index dimension (only 1 repetition anyway).
df_tensorflow_accuracy_50000 = \
    df_tensorflow_accuracy_50000_raw[['accuracy_top1', 'accuracy_top5']] \
    .groupby(level=df_tensorflow_accuracy_50000_raw.index.names[:-1]).min()
# Display all rows and columns.
display_in_full(df_tensorflow_accuracy_50000)

### TensorFlow accuracy on 50,000 images (claimed)

In [ ]:
# TensorFlow accuracy reported with the MobileNets pretrained weights shared on 2017_06_14. Copied from:
# https://github.com/tensorflow/models/blob/1630da3434974e9ad5a0b6d887ac716a97ce03d3/research/slim/nets/mobilenet_v1.md#pre-trained-models
tensorflow_accuracy_50000_table = {
    'v1-1.00-224':[569, 4.24, 70.7, 89.5],
    'v1-1.00-192':[418, 4.24, 69.3, 88.9],
    'v1-1.00-160':[291, 4.24, 67.2, 87.5],
    'v1-1.00-128':[186, 4.24, 64.1, 85.3],
    'v1-0.75-224':[317, 2.59, 68.4, 88.2],
    'v1-0.75-192':[233, 2.59, 67.4, 87.3],
    'v1-0.75-160':[162, 2.59, 65.2, 86.1],
    'v1-0.75-128':[104, 2.59, 61.8, 83.6],
    'v1-0.50-224':[150, 1.34, 64.0, 85.4],
    'v1-0.50-192':[110, 1.34, 62.1, 84.0],
    'v1-0.50-160':[77,  1.34, 59.9, 82.5],
    'v1-0.50-128':[49,  1.34, 56.2, 79.6],
    'v1-0.25-224':[41,  0.47, 50.6, 75.0],
    'v1-0.25-192':[34,  0.47, 49.0, 73.6],
    'v1-0.25-160':[21,  0.47, 46.0, 70.7],
    'v1-0.25-128':[14,  0.47, 41.3, 66.2],
}

In [ ]:
df_tensorflow_accuracy_50000_claimed = pd.DataFrame(
    index=['MACs (million)', 'Parameters (million)', 'accuracy_top1 (%)', 'accuracy_top5 (%)'],
    data=tensorflow_accuracy_50000_table,
).T.sort_index()
accuracy_top1 = df_tensorflow_accuracy_50000_claimed['accuracy_top1 (%)']/100
accuracy_top5 = df_tensorflow_accuracy_50000_claimed['accuracy_top5 (%)']/100
df_tensorflow_accuracy_50000_claimed = df_tensorflow_accuracy_50000_claimed.assign(accuracy_top1=accuracy_top1)
df_tensorflow_accuracy_50000_claimed = df_tensorflow_accuracy_50000_claimed.assign(accuracy_top5=accuracy_top5)
df_tensorflow_accuracy_50000_claimed.index = df_tensorflow_accuracy_50000.index
display_in_full(df_tensorflow_accuracy_50000_claimed)

In [ ]:
# Diff measured as the fraction of correctly predicted images.
df_tensorflow_accuracy_50000_diff = \
    df_tensorflow_accuracy_50000_claimed[['accuracy_top1', 'accuracy_top5']] - \
    df_tensorflow_accuracy_50000[['accuracy_top1', 'accuracy_top5']]
display_in_full(df_tensorflow_accuracy_50000_diff)

In [ ]:
# Diff measured as the number of mispredicted images.
df_tensorflow_accuracy_50000_diff_mispredicted = (df_tensorflow_accuracy_50000_diff) * 50000
df_tensorflow_accuracy_50000_diff_mispredicted.columns = ['mispredicted_top1', 'mispredicted_top5']
display_in_full(df_tensorflow_accuracy_50000_diff_mispredicted)

<a id="experiments_armcl_hikey"></a>
## ArmCL experiments on HiKey

### ArmCL performance (latency)

In [ ]:
df_armcl_performance_raw = get_experimental_results(repo_uoa=armcl_performance_repo_uoa,
                                                    tags='explore-mobilenets-performance', accuracy=False)
# Take the minimum execution time out of several repetitions.
df_armcl_performance = df_armcl_performance_raw.groupby(level=df_armcl_performance_raw.index.names[:-1]).min()
# Display all rows and columns.
display_in_full(df_armcl_performance)

### ArmCL accuracy on 500 images

In [ ]:
df_armcl_accuracy_500_raw = get_experimental_results(repo_uoa=armcl_accuracy_500_repo_uoa,
                                                     tags='explore-mobilenets-accuracy', accuracy=True)
# Extract frame predictionsdf_armcl_accuracy_500_raw
df_armcl_predictions_500 = df_armcl_accuracy_500_raw[['frame_predictions']]
# Reduce the repetition_id index dimension (only 1 repetition anyway).
df_armcl_accuracy_500 = \
    df_armcl_accuracy_500_raw[['accuracy_top1', 'accuracy_top5']] \
    .groupby(level=df_armcl_accuracy_500_raw.index.names[:-1]).min()
# Display all rows and columns.
display_in_full(df_armcl_accuracy_500)

In [ ]:
# Identical accuracy for "18.03" and "dv/dt".
(df_armcl_accuracy_500.loc[hikey_id,'armcl-18.03'] - df_armcl_accuracy_500.loc[hikey_id,'armcl-dv/dt'] == 0).all()

In [ ]:
# Identical accuracy for "18.03" and "18.01".
(df_armcl_accuracy_500.loc[hikey_id,'armcl-18.03'] - df_armcl_accuracy_500.loc[hikey_id,'armcl-18.01'] == 0).all()

In [ ]:
df_armcl_accuracy_500.loc[hikey_id,'armcl-18.03'] - df_armcl_accuracy_500.loc[hikey_id,'armcl-17.12']

In [ ]:
# TODO: Outline into a function for comparing ArmCL and TensorFlow predictions.
df_armcl_predictions = df_armcl_predictions_500
df_tensorflow_predictions = df_tensorflow_predictions_500

tensorflow_lib = 'tensorflow-1.7'
tensorflow_convolution_method = 1

for index, row in df_armcl_predictions.iterrows():
    (platform, lib, model, multiplier, resolution, batch_size, convolution_method, repetition_id) = index
    # For now, only check mispredictions for '18.03' and 'v1-1.00-224'.
    if not lib=='armcl-18.03' or not model=='v1-1.00-224': continue
    tensorflow_index = (platform, tensorflow_lib,  model, multiplier, resolution, batch_size, tensorflow_convolution_method, repetition_id)
    # Extract frame predictions.
    armcl_predictions = row['frame_predictions']
    tensorflow_predictions = df_tensorflow_predictions.loc[tensorflow_index]['frame_predictions']
    # At the very minimum, the frame predictions should be of the same length.
    if len(armcl_predictions) != len(tensorflow_predictions):
        print('[Warning] ArmCL and TensorFlow predictions have different length! Skipping...')
        continue
    # Iterate over the frame predictions.
    for (armcl_prediction, tensorflow_prediction) in zip(armcl_predictions, tensorflow_predictions):
        if(armcl_prediction['accuracy_top1'] != tensorflow_prediction['accuracy_top1']):
            print(index)
            print('ArmCL: '+str(armcl_prediction))
            print('TensorFlow: '+str(tensorflow_prediction))
            print('')

### ArmCL accuracy on 50,000 images

In [ ]:
df_armcl_accuracy_50000_raw = get_experimental_results(repo_uoa=armcl_accuracy_50000_repo_uoa,
                                                       tags='explore-mobilenets-accuracy', accuracy=True)
# Extract frame predictions.
df_armcl_predictions_50000 = df_armcl_accuracy_50000_raw[['frame_predictions']]
# Reduce the repetition_id index dimension (only 1 repetition anyway).
df_armcl_accuracy_50000 = \
    df_armcl_accuracy_50000_raw[['accuracy_top1', 'accuracy_top5']] \
    .groupby(level=df_armcl_accuracy_50000_raw.index.names[:-1]).min()
# Display all rows and columns.
display_in_full(df_armcl_accuracy_50000)

### Plot top 1 accuracy on 50,000 images (using the 'dv/dt' fork as reference lib) vs. performance

In [ ]:
accuracy_metric = 'accuracy_top1'

In [ ]:
df_armcl_performance_accuracy_50000 = merge_performance_accuracy(df_armcl_performance, df_armcl_accuracy_50000,
                                                                 reference_lib='armcl-dv/dt',
                                                                 reference_convolution_method=1)
display_in_full(df_armcl_performance_accuracy_50000)

#### Only "18.03"

In [ ]:
plot(df_armcl_performance_accuracy_50000, libs=['armcl-18.03'], accuracy_metric=accuracy_metric,
     save_fig_name='%s-%s-50000-18_03' % (hikey_id, accuracy_metric+'_'))

#### "dv/dt" vs. "18.03"

In [ ]:
plot(df_armcl_performance_accuracy_50000, libs=['armcl-18.03','armcl-dv/dt'], accuracy_metric=accuracy_metric,
     save_fig_name='%s-%s-50000-dv_dt__18_03' % (hikey_id, accuracy_metric+'_'))

#### "dv/dt" vs. "18.03" vs. "18.01" vs. "17.12"

In [ ]:
plot(df_armcl_performance_accuracy_50000, accuracy_metric=accuracy_metric,
     save_fig_name='%s-%s-50000-dv_dt__18_03__18_01__17_12' % (hikey_id, accuracy_metric+'_'))

### Plot top 1 accuracy on 500 images (using the 'dv/dt' fork as the reference lib) vs. performance

In [ ]:
df_armcl_performance_accuracy_500 = merge_performance_accuracy(df_armcl_performance, df_armcl_accuracy_500,
                                                               reference_lib='armcl-dv/dt',
                                                               reference_convolution_method=1)
display_in_full(df_armcl_performance_accuracy_500)

#### "dv/dt" vs. "18.03" vs. "18.01" vs. "17.12"

In [ ]:
plot(df_armcl_performance_accuracy_500, accuracy_metric=accuracy_metric,
     save_fig_name='%s-%s-500-dv_dt__18_03__18_01__17_12' % (hikey_id, accuracy_metric+'_'))

### Plot top 1 accuracy on 500 images vs. performance

In [ ]:
df_armcl_performance_accuracy_500 = merge_performance_accuracy(df_armcl_performance, df_armcl_accuracy_500)
display_in_full(df_armcl_performance_accuracy_500)

#### "dv/dt" vs. "18.03" vs. "18.01" vs. "17.12"

In [ ]:
plot(df_armcl_performance_accuracy_500, accuracy_metric=accuracy_metric,
     save_fig_name='%s-%s-500-dv_dt__18_03__18_01__17_12' % (hikey_id, accuracy_metric))

<a id="experiments_armcl_tensorflow_hikey"></a>
## ArmCL vs. TensorFlow on HiKey

In [ ]:
df_accuracy_50000 = pd.DataFrame(
    data=[
        df_armcl_accuracy_50000['accuracy_top1'].values,
        df_tensorflow_accuracy_50000['accuracy_top1'].values,
        df_tensorflow_accuracy_50000_claimed['accuracy_top1'].values,        
    ],
    index=[
        'ArmCL 18.03 (measured)',
        'TensorFlow 1.7 (measured)',
        'TensorFlow 1.x (claimed)',
    ],
    columns=df_tensorflow_accuracy_50000_claimed.index.get_level_values(level='model').values
).T.sort_index(ascending=False)
# df_accuracy_50000.index.name = 'model'
if save_tab:
    save_tab_name = 'accuracy_top1-50000'
    save_tab_path = os.path.join(save_tab_dir, '%s.%s' % (save_tab_name, save_tab_ext))
    with open(save_tab_path, 'w') as f: f.write(df_accuracy_50000.to_latex())
display_in_full(df_accuracy_50000)

In [ ]:
df_performance = pd.concat([df_armcl_performance, df_tensorflow_performance])

### Plot top 1 accuracy on 500 images vs. performance

In [ ]:
df_accuracy_500 = pd.concat([df_armcl_accuracy_500, df_tensorflow_accuracy_500])
df_performance_accuracy_500 = merge_performance_accuracy(df_performance, df_accuracy_500)
plot(df_performance_accuracy_500, accuracy_metric=accuracy_metric, save_fig=True,
     save_fig_name='%s-%s-500-dv_dt__18_03__18_01__17_12__tf' % (hikey_id, accuracy_metric))

### Plot top 1 accuracy on 50,000 images vs. performance

In [ ]:
df_accuracy_50000 = pd.concat([df_armcl_accuracy_50000, df_tensorflow_accuracy_50000])
df_performance_accuracy_50000 = merge_performance_accuracy(df_performance, df_accuracy_50000,
                                                           reference_lib='armcl-dv/dt',
                                                           reference_convolution_method=1)

#### "dv/dt" vs. "18.03" vs. "18.01" vs. "17.12"  vs. "tf"

In [ ]:
plot(df_performance_accuracy_50000, accuracy_metric=accuracy_metric, save_fig=True,
     save_fig_name='%s-%s-50000-dv_dt__18_03__18_01__17_12__tf' % (hikey_id, accuracy_metric+'_'))

#### "dv/dt" vs. "18.03" vs. "17.12"  vs. "tf" (no "18.01")

In [ ]:
plot(df_performance_accuracy_50000, libs=['armcl-17.12','armcl-18.03','armcl-dv/dt','tensorflow-1.7'],
     accuracy_metric=accuracy_metric,
     save_fig_name='%s-%s-50000-dv_dt__18_03__17_12__tf' % (hikey_id, accuracy_metric))

<a id="experiments_tensorflow_firefly"></a>
## TensorFlow experiments on Firefly

### TensorFlow accuracy on 500 images

In [ ]:
df_firefly_tensorflow_accuracy_500_raw = get_experimental_results(repo_uoa=firefly_tensorflow_accuracy_500_repo_uoa,
                                                                  tags='explore-mobilenets-accuracy', accuracy=True)
# Extract frame predictions.
df_firefly_tensorflow_predictions_500 = df_firefly_tensorflow_accuracy_500_raw[['frame_predictions']]
# Reduce the repetition_id index dimension (only 1 repetition anyway).
df_firefly_tensorflow_accuracy_500 = \
    df_firefly_tensorflow_accuracy_500_raw[['accuracy_top1', 'accuracy_top5']] \
    .groupby(level=df_firefly_tensorflow_accuracy_500_raw.index.names[:-1]).min()
# Display all rows and columns.
display_in_full(df_firefly_tensorflow_accuracy_500)

In [ ]:
# Check whether TensorFlow accuracy on Firefly is the same as on HiKey. (It's not!)
df_firefly_tensorflow_accuracy_500.loc[firefly_id].loc['tensorflow-1.7'] - \
df_tensorflow_accuracy_500.loc[hikey_id].loc['tensorflow-1.7']

### TensorFlow performance (latency)

In [ ]:
df_firefly_tensorflow_performance_raw = get_experimental_results(repo_uoa=firefly_tensorflow_performance_repo_uoa,
                                                                 tags='explore-mobilenets-performance', accuracy=False)

In [ ]:
# Take the minimum execution time out of several repetitions.
df_firefly_tensorflow_performance_min = \
    df_firefly_tensorflow_performance_raw.groupby(level=df_firefly_tensorflow_performance_raw.index.names[:-1]).min()
# Display all rows and columns.
# display_in_full(df_firefly_tensorflow_performance_min)

In [ ]:
# Take the maximum execution time out of several repetitions.
df_firefly_tensorflow_performance_max = \
    df_firefly_tensorflow_performance_raw.groupby(level=df_firefly_tensorflow_performance_raw.index.names[:-1]).max()
# Set 'convolution_method' to 0 for all rows to reuse the available plotting functionality.
df_firefly_tensorflow_performance_max.index = \
    df_firefly_tensorflow_performance_max.index \
    .set_levels(pd.Int64Index(data=[0]*df_firefly_tensorflow_performance_max.index.size), level='convolution_method')
# Display all rows and columns.
# display_in_full(df_firefly_tensorflow_performance_max)

#### "tf-1.7" vs "tf-1.8" (min/max)

In [ ]:
# accuracy_metric = 'accuracy_top1'

In [ ]:
df_firefly_tensorflow_performance = pd.concat([df_firefly_tensorflow_performance_min, df_firefly_tensorflow_performance_max])
# TODO: Use df_firefly_armcl_accuracy_500 and df_firefly_tensorflow_accuracy_500.
df_accuracy_500 = pd.concat([df_armcl_accuracy_500, df_tensorflow_accuracy_500])
df_firefly_performance_accuracy_500 = merge_performance_accuracy(df_firefly_tensorflow_performance, df_accuracy_500,
                                                                 reference_platform=hikey_id,
                                                                 reference_lib='tensorflow-1.7')
plot(df_firefly_performance_accuracy_500, accuracy_metric=accuracy_metric, platform_id=firefly_id,
     xmin=0, xmax=150.1, xstep=10, save_fig_name='%s-%s-500-tf-min_max' % (firefly_id, accuracy_metric))

In [ ]:
# plot(df_firefly_performance_accuracy_500, platform_id=firefly_id, title=firefly_name,
#      xmin=10., xmax=190.1, xstep=10, ymin=0.35, ymax=.801,
#      accuracy_metric=accuracy_metric, save_fig_name='%s-%s-500-tf-min_max-complete' % (firefly_id, accuracy_metric))

#### "tf-1.7" vs "tf-1.8"

In [ ]:
df_firefly_performance = df_firefly_tensorflow_performance_min
df_firefly_accuracy_500 = df_firefly_tensorflow_accuracy_500
df_firefly_performance_accuracy_500 = merge_performance_accuracy(df_firefly_performance, df_firefly_accuracy_500)
plot(df_firefly_performance_accuracy_500, platform_id=firefly_id, title=firefly_name, xmin=0, xmax=150.1, xstep=10,
     accuracy_metric=accuracy_metric, save_fig_name='%s-%s-500-tf' % (firefly_id, accuracy_metric))

<a id="experiments_armcl_firefly"></a>
## ArmCL experiments on Firefly

### ArmCL performance (latency)

In [ ]:
df_firefly_armcl_performance_raw = get_experimental_results(repo_uoa=firefly_armcl_performance_repo_uoa,
                                                            tags='explore-mobilenets-performance', accuracy=False)
# Take the minimum execution time out of several repetitions.
df_firefly_armcl_performance = \
    df_firefly_armcl_performance_raw .groupby(level=df_firefly_armcl_performance_raw.index.names[:-1]).min()
# Display all rows and columns.
display_in_full(df_firefly_armcl_performance)

### ArmCL accuracy on 500 images

In [ ]:
df_firefly_armcl_accuracy_500_raw = get_experimental_results(repo_uoa=firefly_armcl_accuracy_500_repo_uoa,
                                                             tags='explore-mobilenets-accuracy', accuracy=True)
# Extract frame predictions.
# df_firefly_armcl_predictions_500 = df_firefly_armcl_accuracy_500_raw[['frame_predictions']]
# Reduce the repetition_id index dimension (only 1 repetition anyway).
df_firefly_armcl_accuracy_500 = \
    df_firefly_armcl_accuracy_500_raw[['accuracy_top1', 'accuracy_top5']] \
    .groupby(level=df_firefly_armcl_accuracy_500_raw.index.names[:-1]).min()
# Display all rows and columns.
display_in_full(df_firefly_armcl_accuracy_500)

In [ ]:
# Identical accuracy for "18.03" and "18.01".
(df_firefly_armcl_accuracy_500.loc[firefly_id,'armcl-18.03'] - df_firefly_armcl_accuracy_500.loc[firefly_id,'armcl-18.01'] == 0).all()

In [ ]:
# Non-identical accuracy for "18.03" and "17.12".
df_firefly_armcl_accuracy_500.loc[firefly_id,'armcl-18.03'] - df_firefly_armcl_accuracy_500.loc[firefly_id,'armcl-17.12']

In [ ]:
# Non-identical accuracy for "18.03" on HiKey and "18.03" on Firefly.
df_armcl_accuracy_500.loc[hikey_id,'armcl-18.03'] - df_firefly_armcl_accuracy_500.loc[firefly_id,'armcl-18.03']

In [ ]:
df_firefly_armcl_performance_accuracy_500 = merge_performance_accuracy(df_firefly_armcl_performance, df_firefly_armcl_accuracy_500)
display_in_full(df_firefly_armcl_performance_accuracy_500)

### Plot top 1 accuracy on 500 images vs. performance

In [ ]:
plot(df_firefly_armcl_performance_accuracy_500, accuracy_metric=accuracy_metric, platform_id=firefly_id,
     xmin=0, xmax=150.1, xstep=10, save_fig_name='%s-%s-500-18_03__18_01__17_12' % (firefly_id, accuracy_metric))

<a id="experiments_armcl_tensorflow_firefly"></a>
## ArmCL vs. TensorFlow on Firefly

In [ ]:
df_firefly_tensorflow_performance = df_firefly_tensorflow_performance_min.loc[firefly_id,['tensorflow-1.7'],:]
df_firefly_accuracy_500 = pd.concat([df_firefly_armcl_accuracy_500, df_firefly_tensorflow_accuracy_500])
df_firefly_performance = pd.concat([df_firefly_armcl_performance, df_firefly_tensorflow_performance])
df_firefly_performance_accuracy_500 = merge_performance_accuracy(df_firefly_performance, df_firefly_accuracy_500)
plot(df_firefly_performance_accuracy_500, accuracy_metric=accuracy_metric, platform_id=firefly_id, save_fig=True,
     xmin=0, xmax=150.1, xstep=10, save_fig_name='%s-%s-500-18_03__18_01__17_12__tf' % (firefly_id, accuracy_metric))